In [1]:
from schemas import artist_schema

In [3]:
import findspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, avg, round
from pyspark.sql.types import *

In [4]:
conf = SparkConf()
conf.setAll([
    ("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262"),
    ("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"),
    ("fs.s3a.aws.credentials.provider","com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
])

In [5]:
# Creating a SparkContext object  
findspark.init()
sc = SparkContext.getOrCreate()
# Creating a SparkSession  
spark = SparkSession.builder \
    .config(conf = conf) \
    .appName("awesome-api-potify") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/21 15:36:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/21 15:36:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
s3_path_with_wildcard = 's3a://bucket/User/top/tracks/*.json'

In [7]:
tracks = spark.read.option("multiline","true").json(s3_path_with_wildcard)

24/05/21 15:36:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/05/21 15:36:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
tracks_artist = tracks.select('id', explode("artists").alias("artists"))
tracks_artist = tracks_artist.withColumn('track_id', col('id')).drop('id')

In [17]:
tracks_artist.printSchema()

root
 |-- artists: struct (nullable = true)
 |    |-- external_urls: struct (nullable = true)
 |    |    |-- spotify: string (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- uri: string (nullable = true)



In [12]:
artist = tracks_artist.select('track_id', 'artists.*')
artist.printSchema()

root
 |-- track_id: string (nullable = true)
 |-- external_urls: struct (nullable = true)
 |    |-- spotify: string (nullable = true)
 |-- href: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- uri: string (nullable = true)



In [13]:
artist.show()

+--------------------+--------------------+--------------------+--------------------+-------------------+------+--------------------+
|            track_id|       external_urls|                href|                  id|               name|  type|                 uri|
+--------------------+--------------------+--------------------+--------------------+-------------------+------+--------------------+
|5EvuUq5j67inDYsln...|{https://open.spo...|https://api.spoti...|4GYycScCEPQprmeYJ...|              Yedai|artist|spotify:artist:4G...|
|5FIp4LrsffxiHLhmm...|{https://open.spo...|https://api.spoti...|7qvsLYsYP0MHD7jkd...|         Sara Evans|artist|spotify:artist:7q...|
|5FqYA8KfiwsQvyBI4...|{https://open.spo...|https://api.spoti...|2cnMpRsOVqtPMfq7Y...|          Van Halen|artist|spotify:artist:2c...|
|5GFsQYcVVYRYxTwrf...|{https://open.spo...|https://api.spoti...|4GYycScCEPQprmeYJ...|              Yedai|artist|spotify:artist:4G...|
|5GNwT4AkWakd4RJxy...|{https://open.spo...|https://api.spoti..

24/05/21 17:44:09 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1803773 ms exceeds timeout 120000 ms
24/05/21 17:44:09 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/21 17:44:13 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [44]:
duration = tracks.agg(avg('duration_ms').alias('avg_duration')).withColumn('avg_duration_min', round((col('avg_duration')/60000), 2)).drop('avg_duration').head()[0]

In [46]:
duration

3.22